Загрузим уже имеющиеся наработки по цепи маркова

In [1]:
import re
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
import pandas as pd

txts = ['./external_data/war_and_peace/tom_1.txt', './external_data/war_and_peace/tom_2.txt', './external_data/war_and_peace/tom_3.txt', './external_data/war_and_peace/tom_4.txt']

contents = ""
for txt in txts:
    with open(txt, encoding='utf-8') as stream:
        text = stream.read()
        text = re.sub("[^А-Яа-я| |.|!|?]","",text)
        text = re.sub("[.|!|?]"," ",text)
        text = text.upper()
        contents = contents + " " + text if len(contents)>0 else text

words = []
for word in contents.split(" "):
    if (len(word)>0):
       words.append(word)  
    
RUSSIAN = "АБВГДЕЖЗИЙКЛМНОПРСТУФХЦЧШЩЪЫЬЭЮЯ"
POS = {l: i for i, l in enumerate(RUSSIAN)}
probabilities = np.zeros((len(RUSSIAN), len(RUSSIAN)))
   
def split(s):
    return [char for char in s]
     
for word in words:
    if (len(word)>1):
        for cp, cn in zip(word[:-1], word[1:]):
            probabilities[POS[cp], POS[cn]] += 1
    
probabilities = (probabilities / probabilities.sum(axis=1)[:, None])
  
df = pd.DataFrame(probabilities, index=(c for c in RUSSIAN), columns=(c for c in RUSSIAN))

Добавим методы для определения вероятностей подобранных строк

In [2]:
def get_words(let_f, length, count, let_e=None):
    depth = length - 1 if let_e == None else length - 2
    chains = get_chains(let_f, depth)
    out = []
    for i in range(count):
        out.append((let_f + chains[i][0] + str(let_e or ''), chains[i][1]/depth))
    return out

def takeSecond(elem):
    return elem[1]

def get_chains(let, depth):
    depth_next = depth - 1
    chains = []
    for let_n in RUSSIAN:
        proba = probabilities[POS[let], POS[let_n]]
        if (proba>0.01):
            if(depth_next>0):
                for chain in get_chains(let_n, depth_next):
                    chains.append((let_n+chain[0], proba+chain[1]))
            else:
                chains.append((let_n, proba))
    chains.sort(key=takeSecond, reverse=True)
    return chains

Вычислим 30 первых вероятных строк из 4-х символов, которая начинается П, а кончается на Р

In [3]:
out = get_words("П",4,30,"Р")

Выведим строки на экран

In [4]:
print(out)

[('ПРАР', 0.25248155485854795), ('ПОВР', 0.24313794125321347), ('ПОТР', 0.2383136352932057), ('ПОСР', 0.238046696834654), ('ПОЛР', 0.23678237922642256), ('ПОНР', 0.23601311112314166), ('ПОРР', 0.23351844989231269), ('ПРОР', 0.23017188360589147), ('ПОМР', 0.22465366635513545), ('ПОГР', 0.22378732972147208), ('ПОДР', 0.22328742679000246), ('ПРЕР', 0.22055757450407437), ('ПОЙР', 0.2165338837886435), ('ПОБР', 0.2145682460483989), ('ПРИР', 0.2087417869035334), ('ПОЕР', 0.20824665800724182), ('ПОЖР', 0.20379363917594695), ('ПОКР', 0.202963703241177), ('ПОЧР', 0.20103689291308535), ('ПОПР', 0.20086944969817563), ('ПОШР', 0.19967065334795236), ('ПОЗР', 0.198704821470647), ('ПОИР', 0.19797680749277863), ('ПРУР', 0.17823631117063934), ('ПРЫР', 0.1608888639705336), ('ПРЯР', 0.15303370396931337), ('ПРЬР', 0.1522456458462136), ('ПРНР', 0.14770541420796787), ('ПРТР', 0.14669364926282688), ('ПЕНР', 0.12485774220315793)]


Запишем полученный результат в файл

In [5]:
out_df = pd.DataFrame(out, columns = ['STR','PROBA'])
out_df.to_csv('./processed/res_strings.csv')